In [2]:
import pandas as pd
path='../../data/news_info_new_id.txt'
doc_info=pd.read_table(path,header=None)
doc_info

,0,1,2,3,4,5,6
0,1,合集|外交部每日答问,1.624890e+12,0.0,NaN,NaN,NaN
1,2,汽车点评网友“毛蒙”分享的小视频“,1.563405e+12,0.0,汽车,汽车/其他,https:8.135272
2,3,"抖音爆红车模小姐姐,可爱如神仙,如今大火的她还玩起这个!",1.567068e+12,3.0,娱乐,娱乐/其他,"初恋:11.052198,吊坠:9.525825,吊坠^^耳环:7.982950,妆容:10..."
3,4,"一夜能挣30万的网红车模,她的捞金神器竟是这个!",1.569240e+12,3.0,娱乐,娱乐/其他,"主播:10.299306,发展:7.355088,可爱:8.705284,清纯:10.108..."
4,5,"男子购买假号牌被查,后备箱找到真车牌,车主直言不愿太高调",1.571271e+12,3.0,汽车,汽车/用车,"买车:9.357286,交警:8.964902,交通:7.790495,交通^^秩序:10...."
...,...,...,...,...,...,...,...
347461,347462,发病率高达40%就诊率仅9.4%!女性羞于启齿的“社交癌”你了解多少?,1.625068e+12,3.0,健康,健康/疾病防护治疗及西医用药,"ics:8.456915,oab:10.391586,oab^^患者:7.256806,oa..."
347462,347463,"50分惨败!昔日非洲篮坛霸主彻底没落,曾是中国男篮老对手",1.625068e+12,6.0,体育,体育/中国篮球,"三分球:8.266778,世界大赛:8.078333,世界杯:10.498599,世锦赛:1..."
347463,347464,马上到!宝鸡雷电预警……做好防护,1.625042e+12,4.0,天气,天气/天气预报,"冰雹:10.477194,凤县:15.765309,凤翔:11.166350,千阳县:16...."
347464,347465,"《好声音》史上最讽刺的3场比赛:赢家销声匿迹,淘汰者越来越火",1.625068e+12,16.0,综艺,综艺/内地综艺,"90后:7.156848,ain:9.383776,中国^^好^^声音:9.787591,代..."


In [3]:
text=[doc_info.loc[i][1] for i in range(len(doc_info))]
text[0:3]

['合集|外交部每日答问', '汽车点评网友“毛蒙”分享的小视频“', '抖音爆红车模小姐姐,可爱如神仙,如今大火的她还玩起这个!']

In [4]:
import jieba
def tokenize(text,stop):
    """
    带有语料清洗功能的分词函数
    """
#     text = re.sub("\{%.+?%\}", " ", text)           # 去除 {%xxx%} (地理定位, 微博话题等)
#     text = re.sub("@.+?( |$)", " ", text)           # 去除 @xxx (用户名)
#     text = re.sub("【.+?】", " ", text)              # 去除 【xx】 (里面的内容通常都不是用户自己写的)
#     icons = re.findall("\[.+?\]", text)             # 提取出所有表情图标
#     text = re.sub("\[.+?\]", "IconMark", text)      # 将文本中的图标替换为`IconMark`

    tokens = []
    s=''
    for k, w in enumerate(jieba.lcut(text)):
        w = w.strip()#移除字符串头尾指定的字符
        if w in stop:
            continue
        if w.encode('utf-8').isalnum() is True:
          # print(w,w.encode('utf-8').isalnum())
          continue
        tokens.append(w)
        s+=w
        s+=" "
    return tokens,s
stopwords = []
with open("stop_wb.txt", "r", encoding="utf8") as f:
    for w in f:
        stopwords.append(w.strip())
tokenize(text[0],stopwords)

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Dumping model to file cache /tmp/jieba.cache
Dump cache file failed.
Traceback (most recent call last):
  File "/data/fuwen/anaconda3/envs/bj2/lib/python3.7/site-packages/jieba/__init__.py", line 154, in initialize
    _replace_file(fpath, cache_file)
PermissionError: [Errno 1] Operation not permitted: '/tmp/tmpe9h4gqb_' -> '/tmp/jieba.cache'
Loading model cost 0.985 seconds.
Prefix dict has been built successfully.


(['合集', '外交部', '每日', '答问'], '合集 外交部 每日 答问 ')

In [5]:
text_clear=[]
for i in text:
    if isinstance(i, str):
        tokens,s=tokenize(i,stopwords)
        text_clear.append(tokens)



In [6]:
print(len(text_clear),text_clear[0:3])

347465 [['合集', '外交部', '每日', '答问'], ['汽车', '点评', '网友', '毛蒙', '分享', '的', '小视频'], ['抖音', '爆红', '车模', '小姐姐', '可爱', '如', '神仙', '如今', '大火', '的', '她', '还', '玩', '起', '这个']]


In [7]:
from gensim.models import Word2Vec
from gensim.models.word2vec import LineSentence
import multiprocessing
model = Word2Vec(text_clear, vector_size=300, window=5, min_count=3, workers=multiprocessing.cpu_count(), sg=0)
# sg: 如果是0， 则是CBOW模型，是1则是Skip-Gram模型，默认是0即CBOW模型
model.save('v300.model')

In [8]:
from gensim.models import Word2Vec
model0 = Word2Vec.load('v300.model')
print(model0.wv['外交部'])

[ 0.75593     0.05107793 -0.2837511   0.48917556  0.38567272  0.6176994
  0.05105739 -0.34700665 -0.08905835  0.3875873   0.6692712  -0.58959836
 -0.10618126  0.25350207 -0.23971455  0.40083104 -0.01862572  0.78874147
 -0.07100475  0.7620168   0.35901362  0.06328573 -0.7228775  -0.08961568
  0.8541479  -0.7439106  -0.36293325  0.41005948 -0.4199395   0.07829548
  0.5459787   0.57156307  0.25839078  0.00983621 -0.11083393  0.10585054
 -0.21391995  0.25603563  0.15335114  0.3882821   0.2050197   0.06699337
  0.1297318  -0.34697098 -0.5120233   1.0082737   0.5169733   0.5682281
  0.08735414 -0.21545075 -0.3395283  -0.18798013  0.03458227  0.04598101
  0.24307789  0.34595656  0.6059055   0.38666898 -0.17747706  0.49454877
 -0.42833367  0.72942895  0.20056334 -0.17227547  0.867699    0.38983443
  0.11707778  1.1334654  -0.37156054 -0.05798044 -0.18487273  0.04766329
 -0.33741626 -0.19577596  0.49466515 -0.09776149 -0.08037123 -0.22711466
 -0.26391026  0.40661964 -0.97296417 -0.26520267  0.0

In [9]:
def get_similar_words_str(w, model, topn = 10):
    result_words = get_similar_words_list(w, model)    
    return str(result_words)


def get_similar_words_list(w, model, topn = 10):
    result_words = []
    try:
        similary_words = model.wv.most_similar(w, topn=10)
        print(similary_words)
        for (word, similarity) in similary_words:
            result_words.append(word)
        print(result_words)
    except:
        print("There are some errors!" + w)

    return result_words

In [10]:
get_similar_words_list('可爱',model0)

[('精致', 0.7862719297409058), ('温柔', 0.7857161164283752), ('看着', 0.7828270792961121), ('接地', 0.7825865745544434), ('浪漫', 0.7819203734397888), ('漂亮', 0.7760711908340454), ('迷人', 0.7710559368133545), ('气', 0.7656367421150208), ('安静', 0.7632823586463928), ('小姐姐', 0.7630001902580261)]
['精致', '温柔', '看着', '接地', '浪漫', '漂亮', '迷人', '气', '安静', '小姐姐']


['精致', '温柔', '看着', '接地', '浪漫', '漂亮', '迷人', '气', '安静', '小姐姐']